# Mission to Mars

Nasa Web Scraping

In [109]:
# import dependencies 
from bs4 import BeautifulSoup as bs
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import pandas as pd
import time as time
import re

In [110]:
# go to the directory
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

### Nasa Mars News

In [111]:
# browse the web page
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
time.sleep(1)
nasa_html = browser.html
nasa_soup = bs(nasa_html, 'html.parser')

In [112]:
# print the news titles
news_title = nasa_soup.find("li", class_="slide").find("div", class_="content_title").text
news_p = nasa_soup.find("li", class_="slide").find("div", class_="article_teaser_body").text
print(news_title)
print(news_p)

How NASA's Mars Helicopter Will Reach the Red Planet's Surface
The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.


### JPL Mars Space Images

In [99]:
# visit the jpl image source
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(jpl_url)

In [100]:
# jpl html
jpl_html = browser.html
jpl_soup = bs(jpl_html, 'html.parser')

In [101]:
# find the featured image and print
featured_img_base = "https://www.jpl.nasa.gov"
featured_img_url_raw = jpl_soup.find("div", class_="carousel_items").find("article")["style"]
featured_img_url = featured_img_url_raw.split("'")[1]
featured_img_url = featured_img_base + featured_img_url
print(featured_img_url)

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19113-1920x1200.jpg'

### Mars Weather

In [102]:
# to said person reading this, this part too me so long to figure out, like its so stupid easy
# and yet it took me SO LONG, I tried like 50 different variations of this

In [103]:
# find the weather and print it
# visit the twitter source
mars_twitter_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(mars_twitter_url)

# wait a bit cause idk why it just doesnt work thanks america
time.sleep(5)

# convert html to bs
twitter = browser.html
soup = bs(twitter,"lxml")

# use this if you need to print to console (gross!)
# print(weather_soup.prettify())

mars_insight = soup.find(text=re.compile('InSight sol'))
mars_weather = mars_insight.replace('InSight sol', 'Sol', 1)
print(mars_weather)

Sol 559 (2020-06-22) low -90.7ºC (-131.2ºF) high -2.8ºC (27.0ºF)
winds from the SW at 4.5 m/s (10.0 mph) gusting to 14.8 m/s (33.1 mph)
pressure at 7.60 hPa


### Mars Facts

In [104]:
# facts url
facts_url = "https://space-facts.com/mars/"

# make into a table
tables = pd.read_html(facts_url)
mars_df = tables[0]

# assign column values and set index
mars_df.columns = ['Description','Value']
mars_df.set_index('Description', inplace=True)

# convert to html string
mars_df.to_html()
data = mars_df.to_dict(orient='records') 

# print results
mars_df

,Value
Description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.39 × 10^23 kg (0.11 Earths)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.38 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-87 to -5 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


### Mars Hemispheres 

In [105]:
# hemispheres website
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

# HTML Object and parse said object
html_hemispheres = browser.html
hemisphere_soup = bs(html_hemispheres, 'html.parser')

# retrieve all items for it
items = hemisphere_soup.find_all('div', class_='item')

# make an empty list for image urls
hemisphere_image_urls = []
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# loopity loop through the items previously stored
for i in items: 
    title = i.find('h3').text
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    browser.visit(hemispheres_main_url + partial_img_url)
    partial_img_html = browser.html
     
    soup = bs( partial_img_html, 'html.parser')
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})

# print result
print(hemisphere_image_urls)


[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


In [106]:
results = {
        'news_title' : news_title,
        'news_p': news_p,
        'featured_image_url': featured_img_url,
        'mars_weather' : mars_weather,
        'hemisphere_image_urls': hemisphere_image_urls
    }

print(results)

{'news_title': "How NASA's Mars Helicopter Will Reach the Red Planet's Surface", 'news_p': 'The small craft will seek to prove that powered, controlled flight is possible on another planet. But just getting it onto the surface of Mars will take a whole lot of ingenuity.', 'featured_image_url': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA19113-1920x1200.jpg', 'mars_weather': 'Sol 559 (2020-06-22) low -90.7ºC (-131.2ºF) high -2.8ºC (27.0ºF)\nwinds from the SW at 4.5 m/s (10.0 mph) gusting to 14.8 m/s (33.1 mph)\npressure at 7.60 hPa', 'hemisphere_image_urls': [{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astrogeology.usgs.g

In [113]:
browser.quit()